<a href="https://colab.research.google.com/github/iansilva2305/DattingApp/blob/master/ingesta_pubsub_cripto_armandoita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install google-cloud
#!pip install google-cloud-pubsub

In [9]:
#librerias

import os
import json
import requests

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from time import sleep
from google.cloud import pubsub_v1

In [3]:
#1. Credenciales
#1.1 Credenciales de Google Cloud Platform
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/focused-evening-419322-4c972c28dfa1.json'

In [4]:
#1.2 (85c5)Credenciales del API de coinmarketcap.com
ACCESS_TOKEN = '794c54f8-7a97-402d-9dc0-5035357c7503'

In [5]:
# 2. Configurando SDK de Google con parametros del Proyecto
project_id = "focused-evening-419322"
topic_id = "topic-cripto-aris"
publisher = pubsub_v1.PublisherClient()
project_path = f"projects/{project_id}"
topic_path = publisher.topic_path(project_id, topic_id)

In [7]:
# https://coinmarketcap.com/api/documentation/v1/

#3.1 Configurando parametros de request para coinmarketcap
url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
  'slug':'bitcoin',
  'convert':'USD'
}

headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': ACCESS_TOKEN,
}

session = Session()
session.headers.update(headers)

In [ ]:
while True:
  #3.2 solicitando la información al APi de Coinmarketcap
  response = requests.request("GET", url, headers=headers)
  try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
  #4. transformandoo la data para enviar por el API
  cryto_array = []
  crypto = {}

  if data['data'].get('1') is not None:

      crypto['nombre'] = data['data']['1']['name']
      crypto['slug'] = data['data']['1']['slug']
      crypto['fecha_creacion'] = data['data']['1']['date_added']
      crypto['ultima_actualizacion'] = data['data']['1']['last_updated']
      crypto['precio_dolares'] = data['data']['1']['quote']['USD']['price']
      crypto['porcentaje_cambio_1h'] = data['data']['1']['quote']['USD']['percent_change_1h']
      crypto['porcentaje_cambio_24h'] = data['data']['1']['quote']['USD']['percent_change_1h']
      crypto['porcentaje_cambio_7d'] = data['data']['1']['quote']['USD']['percent_change_1h']

      cryto_array.append(crypto)

  #5. Enviando la información al servicio de Pub/Sub de GCP.
  future = publisher.publish(topic_path, json.dumps(cryto_array).encode("utf-8"))
  print(cryto_array)
  print(future.result())

  sleep(60)

[{'nombre': 'Bitcoin', 'slug': 'bitcoin', 'fecha_creacion': '2010-07-13T00:00:00.000Z', 'ultima_actualizacion': '2024-07-02T20:11:00.000Z', 'precio_dolares': 61894.91357538907, 'porcentaje_cambio_1h': -0.37201999, 'porcentaje_cambio_24h': -0.37201999, 'porcentaje_cambio_7d': -0.37201999}]
11645360500035565
[{'nombre': 'Bitcoin', 'slug': 'bitcoin', 'fecha_creacion': '2010-07-13T00:00:00.000Z', 'ultima_actualizacion': '2024-07-02T20:12:00.000Z', 'precio_dolares': 61900.0113068211, 'porcentaje_cambio_1h': -0.34097177, 'porcentaje_cambio_24h': -0.34097177, 'porcentaje_cambio_7d': -0.34097177}]
11646055268687779
[{'nombre': 'Bitcoin', 'slug': 'bitcoin', 'fecha_creacion': '2010-07-13T00:00:00.000Z', 'ultima_actualizacion': '2024-07-02T20:13:00.000Z', 'precio_dolares': 61906.47070866806, 'porcentaje_cambio_1h': -0.30671429, 'porcentaje_cambio_24h': -0.30671429, 'porcentaje_cambio_7d': -0.30671429}]
11645564818813895
[{'nombre': 'Bitcoin', 'slug': 'bitcoin', 'fecha_creacion': '2010-07-13T00:00